In [ ]:
import time

# 对上一个文档的代码进行修改

`yield from` 全部改为 `await`

添加了 `await` 的函数使用 `async def` 定义

`YieldFromAble` 的 `__iter__` 方法改为 `__await__` 方法

`YieldFromAble` 改名 `Awaitable`

修改 `Task`

In [13]:
# 定义辅助 class, 用来包裹 yield 内容, 实现 yield from 调用原本 yield 的内容
class Awaitable:
    def __init__(self, obj):
        self.value = obj

    def __await__(self):
        # yield self.value 改为 yield self.value
        yield self

In [ ]:
async def small_step():
    print("         sleeping for 2 seconds")
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()

    await Awaitable((time.sleep, 2))

    t2 = time.time()
    assert t2 - t1 > 2, "睡眠时间不足"
    print("         working on small step")
    return 123

In [15]:
async def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")

    small_result = await small_step()

    print("     end small_step")
    return small_result * 1000

In [16]:
async def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_result = await big_step()

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [19]:
class Task:
    def __init__(self, coro) -> None:
        self.coro = coro
        self._done = False
        self._result = None

    def run(self):
        print("----------")
        if not self._done:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                # 处理协程返回值
                self.result = e.value
                self._done = True
            else:
                print(f"     got {x}")
                # 上面改造 Awaitable yield self 之后，可以保证这里的 x 一定是 Awaitable 类型
                assert isinstance(x, Awaitable)

        else:
            print("Task is done")
        print("----------")


In [20]:
t = Task(one_task())
t.run()

# 在这里等待2秒
# 这里调用返回的函数
for _ in range(10):
    print("do something ...")
    time.sleep(0.2)

t.run()


----------
begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got <__main__.Awaitable object at 0x000001C6DA72EBD0>
----------
do something ...
do something ...
do something ...
do something ...
do something ...
do something ...
do something ...
do something ...
do something ...
do something ...
----------
         working on small step
     end small_step
     end big_step with 123000
end task
----------
